In [1]:
#import library
import collections
import torch
import matplotlib.pyplot as plt
import PIL
import random
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import copy
import cv2
import collections
import pathlib
import numpy as np
import cv2
import os
import sys
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F



In [2]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(),
    transforms.ToTensor(),         
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

In [3]:
cwd_path = pathlib.Path(os.getcwd())

In [4]:

img_dir = os.path.join(cwd_path, 'data')
img_dir

'c:\\Users\\Lenovo\\Videos\\Imperial college\\week 25\\Project2\\data'

In [5]:
#Load image data
def load_images(data_dir, transform= transform):
    dataset = datasets.ImageFolder(data_dir, transform)
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    sample = train_dataset[0][0]
    samplePilImg = transforms.ToPILImage()(sample)
    samplePilImg.show()
    return train_dataset, val_dataset, test_dataset

In [6]:
train_dataset, val_dataset, test_dataset = load_images(img_dir, transform)
len(train_dataset)

780

In [7]:
y_train=[y for _,y in train_dataset]
counter_train=collections.Counter(y_train)
counter_train

c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Counter({0: 434, 1: 346})

In [8]:
y_val=[y for _,y in val_dataset]
counter_val=collections.Counter(y_val)
counter_val

Counter({0: 52, 1: 45})

In [9]:
y_test=[y for _,y in test_dataset]
counter_test=collections.Counter(y_test)
counter_test

Counter({0: 56, 1: 43})

In [10]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [11]:
# = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
img , label = next(iter(train_loader))
img.shape

torch.Size([64, 3, 224, 224])

In [14]:
%pip install bayesian_optimization
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from functools import partial
from bayes_opt import BayesianOptimization

class CNNModel(nn.Module):
    def __init__(self, image_shape, **kwargs):
        super(CNNModel, self).__init__()
        layers = []
        print(f"Initializing CNNModel with kwargs: {kwargs}")
        layers.append(nn.Conv2d(3, kwargs['kernels'], kernel_size=(kwargs['kernel_size'], kwargs['kernel_size']), groups=1))
        for _ in range(kwargs['conv_layers']):
            layers.append(nn.Conv2d(kwargs['kernels'], kwargs['kernels'], kernel_size=(kwargs['kernel_size'], kwargs['kernel_size']), groups=1))
            if kwargs['maxpooling'] == 1:
                layers.append(nn.MaxPool2d(kernel_size=(2, 2)))
            if kwargs['dropout_cnn'] == 1:
                layers.append(nn.Dropout(kwargs['dropout_perc_cnn']))
        layers.append(nn.Flatten())
        
        # Calculate the size of the flattened layer
        with torch.no_grad():
            dummy_input = torch.zeros(1, *image_shape)
            dummy_output = nn.Sequential(*layers)(dummy_input)
            flattened_size = dummy_output.numel()
        
        for _ in range(kwargs['layers']):
            layers.append(nn.Linear(flattened_size if _ == 0 else kwargs['neurons'], kwargs['neurons']))
            if kwargs['dropout'] == 1:
                layers.append(nn.Dropout(kwargs['dropout_perc']))
        layers.append(nn.Linear(kwargs['neurons'], 1))  # Binary classification
        layers.append(nn.Sigmoid())  # Use sigmoid activation
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def fit_with(kernels, kernel_size, conv_layers, maxpooling, dropout_cnn, dropout_perc_cnn, layers, neurons, dropout, dropout_perc, input_shape=None, verbose=1):
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    kwargs = {
        'kernels': int(kernels),
        'kernel_size': int(kernel_size),
        'conv_layers': int(conv_layers),
        'maxpooling': int(maxpooling),
        'dropout_cnn': int(dropout_cnn),
        'dropout_perc_cnn': float(dropout_perc_cnn),
        'layers': int(layers),
        'neurons': int(neurons),
        'dropout': int(dropout),
        'dropout_perc': float(dropout_perc)
    }
    print(f"Calling CNNModel with kwargs: {kwargs}")
    net = CNNModel(input_shape[1:], **kwargs)  # Adjust input_shape to remove the batch dimension
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        model.train()
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        train_loss, train_correct = 0, 0

        for batch, (X, y) in enumerate(train_loader):
            X = X.squeeze(0)  # Remove the extra dimension
            y_pred = model(X).squeeze()
            loss = loss_fn(y_pred, y.float())
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            y_pred_class = (y_pred > 0.5).float()
            train_correct += (y_pred_class == y).sum().item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        train_acc = train_correct / len(train_loader.dataset)
        train_accuracies.append(train_acc)
        
    
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = net(inputs)
            predicted = (outputs > 0.5).float()  # Convert to binary predictions
            total += labels.size(0)
            correct += (predicted == labels.float()).sum().item()
    
    accuracy = 100 * correct / total
    #print(f"Validation Accuracy: {accuracy:.2f}%")
    return accuracy

input_shape = img.shape  # img.shape = [4, 3, 224, 224]
verbose = 1
fit_with_partial = partial(fit_with, input_shape=input_shape, verbose=verbose)
pbounds = {
    'kernels': (6, 64),
    'kernel_size': (3, 5),
    'conv_layers': (1, 3),
    'maxpooling': (0, 1),
    'dropout_cnn': (0, 1),
    'dropout_perc_cnn': (0.1, 0.5),
    'layers': (1, 3),
    'neurons': (64, 500),
    'dropout': (0, 1),
    'dropout_perc': (0.1, 0.5)
}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)
optimizer.maximize(init_points=5, n_iter=6)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))
    
#print the result of the best model
print(optimizer.max)

You should consider upgrading via the 'c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
|   iter    |  target   | conv_l... |  dropout  | dropou... | dropou... | dropou... | kernel... |  kernels  |  layers   | maxpoo... |  neurons  |
-------------------------------------------------------------------------------------------------------------------------------------------------
Calling CNNModel with kwargs: {'kernels': 16, 'kernel_size': 3, 'conv_layers': 1, 'maxpooling': 0, 'dropout_cnn': 0, 'dropout_perc_cnn': 0.15870235632684523, 'layers': 1, 'neurons': 298, 'dropout': 0, 'dropout_perc': 0.22093302905273593}
Initializing CNNModel with kwargs: {'kernels': 16, 'kernel_size': 3, 'conv_layers': 1, 'maxpooling': 0, 'dropout_cnn': 0, 'dropout_perc_cnn': 0.15870235632684523, 'layers': 1, 'neurons': 298, 'dropout': 0, 'dropout_perc': 0.22093302905273593}
Epoch 1, Batch 1, Loss: 0.6946
Epoch 1, Batch 2, Loss: 37.0538


c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1, Batch 3, Loss: 40.9049
Epoch 1, Batch 4, Loss: 32.8125
Epoch 1, Batch 5, Loss: 43.7500
Epoch 1, Batch 6, Loss: 25.4017
Epoch 1, Batch 7, Loss: 43.7500
Epoch 1, Batch 8, Loss: 40.6250
Epoch 1, Batch 9, Loss: 35.9375
Epoch 1, Batch 10, Loss: 29.6875
Epoch 1, Batch 11, Loss: 42.1875
Epoch 1, Batch 12, Loss: 26.5625
Epoch 1, Batch 13, Loss: 33.3333
Epoch 2, Batch 1, Loss: 25.9806
Epoch 2, Batch 2, Loss: 39.0625
Epoch 2, Batch 3, Loss: 42.1875
Epoch 2, Batch 4, Loss: 23.4375
Epoch 2, Batch 5, Loss: 34.3750
Epoch 2, Batch 6, Loss: 42.1875
Epoch 2, Batch 7, Loss: 34.3750
Epoch 2, Batch 8, Loss: 41.2009
Epoch 2, Batch 9, Loss: 42.1875
Epoch 2, Batch 10, Loss: 35.9375
Epoch 2, Batch 11, Loss: 32.8125
Epoch 2, Batch 12, Loss: 35.9375
Epoch 2, Batch 13, Loss: 16.6667
| 1         | 2.792e+03 | 1.834     | 0.7203    | 0.0001144 | 0.2209    | 0.1587    | 3.185     | 16.8      | 1.691     | 0.3968    | 298.9     |
Calling CNNModel with kwargs: {'kernels': 30, 'kernel_size': 4, 'conv_layers':

c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 1, Batch 10, Loss: 53.1250
Epoch 1, Batch 11, Loss: 62.5000
Epoch 1, Batch 12, Loss: 57.8125
Epoch 1, Batch 13, Loss: 58.3333
Epoch 2, Batch 1, Loss: 54.6875
Epoch 2, Batch 2, Loss: 62.5000
Epoch 2, Batch 3, Loss: 46.8750
Epoch 2, Batch 4, Loss: 45.3125
Epoch 2, Batch 5, Loss: 53.1250
Epoch 2, Batch 6, Loss: 56.2500
Epoch 2, Batch 7, Loss: 62.5000
Epoch 2, Batch 8, Loss: 57.8125
Epoch 2, Batch 9, Loss: 59.3750
Epoch 2, Batch 10, Loss: 56.2500
Epoch 2, Batch 11, Loss: 62.5000
Epoch 2, Batch 12, Loss: 53.1250
Epoch 2, Batch 13, Loss: 33.3333
| 9         | 2.554e+03 | 2.287     | 1.0       | 0.3075    | 0.5       | 0.1       | 3.573     | 15.35     | 1.0       | 0.03369   | 298.8     |
Calling CNNModel with kwargs: {'kernels': 16, 'kernel_size': 3, 'conv_layers': 1, 'maxpooling': 0, 'dropout_cnn': 0, 'dropout_perc_cnn': 0.11155468484884645, 'layers': 2, 'neurons': 103, 'dropout': 0, 'dropout_perc': 0.19477098919671654}
Initializing CNNModel with kwargs: {'kernels': 16, 'kernel_size'

c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1, Batch 5, Loss: 16.9147
Epoch 1, Batch 6, Loss: 32.0559
Epoch 1, Batch 7, Loss: 34.7612
Epoch 1, Batch 8, Loss: 53.1262
Epoch 1, Batch 9, Loss: 37.9858
Epoch 1, Batch 10, Loss: 55.5311
Epoch 1, Batch 11, Loss: 44.4776
Epoch 1, Batch 12, Loss: 51.6574
Epoch 1, Batch 13, Loss: 63.8893
Epoch 2, Batch 1, Loss: 52.8100
Epoch 2, Batch 2, Loss: 48.6331
Epoch 2, Batch 3, Loss: 46.5444
Epoch 2, Batch 4, Loss: 48.5164
Epoch 2, Batch 5, Loss: 56.6502
Epoch 2, Batch 6, Loss: 39.2954
Epoch 2, Batch 7, Loss: 37.9493
Epoch 2, Batch 8, Loss: 39.5769
Epoch 2, Batch 9, Loss: 27.0082
Epoch 2, Batch 10, Loss: 28.3219
Epoch 2, Batch 11, Loss: 38.3831
Epoch 2, Batch 12, Loss: 31.6910
Epoch 2, Batch 13, Loss: 41.6667
| 10        | 2.734e+03 | 1.163     | 0.5053    | 0.7729    | 0.1948    | 0.1116    | 3.256     | 16.95     | 2.217     | 0.4944    | 103.8     |
Calling CNNModel with kwargs: {'kernels': 51, 'kernel_size': 3, 'conv_layers': 1, 'maxpooling': 0, 'dropout_cnn': 0, 'dropout_perc_cnn': 0.237

c:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1, Batch 6, Loss: 37.5000
Epoch 1, Batch 7, Loss: 42.1875
Epoch 1, Batch 8, Loss: 39.0625
Epoch 1, Batch 9, Loss: 39.0625
Epoch 1, Batch 10, Loss: 42.1875
Epoch 1, Batch 11, Loss: 28.1250
Epoch 1, Batch 12, Loss: 29.6875
Epoch 1, Batch 13, Loss: 41.6667
Epoch 2, Batch 1, Loss: 45.3125
Epoch 2, Batch 2, Loss: 31.2500
Epoch 2, Batch 3, Loss: 43.7500
Epoch 2, Batch 4, Loss: 39.0625
Epoch 2, Batch 5, Loss: 42.1875
Epoch 2, Batch 6, Loss: 30.4585
Epoch 2, Batch 7, Loss: 39.0625
Epoch 2, Batch 8, Loss: 40.6250
Epoch 2, Batch 9, Loss: 42.1875
Epoch 2, Batch 10, Loss: 29.6875
Epoch 2, Batch 11, Loss: 29.6875
Epoch 2, Batch 12, Loss: 32.8125
Epoch 2, Batch 13, Loss: 41.6667
| 11        | 2.845e+03 | 1.83      | 0.9601    | 0.6484    | 0.306     | 0.238     | 3.787     | 51.08     | 1.179     | 0.7387    | 194.0     |
Iteration 0: 
	{'target': 2791.7525773195875, 'params': {'conv_layers': 1.834044009405148, 'dropout': 0.7203244934421581, 'dropout_cnn': 0.00011437481734488664, 'dropout_perc